In [1]:
import nbimporter
from ColumnTransformers import *
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import json 

In [2]:
def XgboostSearch(scale_pos_weight,X,y,model__early_stopping=15): 
    param_grid_xgb = {
        'n_estimators': [100, 300,500],  
        'learning_rate': [0.001,0.01],  
        'max_depth': [2,3,4], 
        'subsample': [0.7, 1.0],  
        'colsample_bytree': [0.6, 1.0],  
        'gamma': [0, 0.1, 0.5 ], 
        'reg_lambda': [0,1, 4], 
        'scale_pos_weight':[scale_pos_weight], # weight = 1 if negstive else wegiht=scale_pos_weight
        }  
    grid_search_XgBoost = GridSearchgGradient(XGBClassifier(eval_metric='auc',random_state=42, verbosity=0,
                early_stopping_rounds=model__early_stopping, n_jobs=-1),param_grid_xgb,X,y)
    return grid_search_XgBoost


def LightgbmSearch(X,y,model__early_stopping=15):
    param_grid_lgb = {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.001, 0.01],
        'max_depth': [2, 3, 4],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'min_split_gain': [0, 0.1, 0.5],   # similar to gamma from XGB
        'reg_lambda': [0, 1, 4], 
        'is_unbalance':[True] }

    grid_search_LightGBM = GridSearchgGradient(LGBMClassifier(eval_metric='auc',random_state=42 , 
                                                             early_stopping_rounds=model__early_stopping, 
                                                             n_jobs=-1,verbosity=0),param_grid_lgb,X,y)
    return grid_search_LightGBM

def AdaSearch(ModelPipeline) : 
    param_grid_ada = {
    'model__n_estimators': [100,200], 
    'model__learning_rate': [0.6,1,2],  # before amount of say
    'model__estimator': [DecisionTreeClassifier(max_depth=2), 
                          DecisionTreeClassifier(max_depth=3),
                          DecisionTreeClassifier(max_depth=5)]
    }
    grid_search_ada = GridSearchCV(ModelPipeline, param_grid_ada, cv=5, scoring='roc_auc', n_jobs=3)
    return grid_search_ada

In [3]:
def kfold_split_np(X, y, n_splits=5, shuffle=True, random_state=42):
    # Create copies to avoid modifying original arrays
    X_copy = X.copy()
    y_copy = y.copy()
    n_samples = X_copy.shape[0]
 
    indices = np.arange(n_samples)
  
    if shuffle:
        rng = np.random.default_rng(seed=random_state)
        rng.shuffle(indices)

    fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=int)
    fold_sizes[:n_samples % n_splits] += 1
    
    splits = []
    current = 0
    for fold_size in fold_sizes:
        start = current
        stop = current + fold_size
        test_indices = indices[start:stop]
        train_indices = np.concatenate((indices[:start], indices[stop:]))
     
        X_train, X_test = X_copy[train_indices], X_copy[test_indices]
        y_train, y_test = y_copy[train_indices], y_copy[test_indices]
        
        splits.append((X_train, X_test, y_train, y_test))
        current = stop
        
    return splits


In [4]:
def GridSearchgGradient(Model,params,df_X,y, n_splits=5):
    BestModelParams=None 
    BestAucScore=float("-inf")
    Lr=PipeLineGradient() 
    Lr.fit(df_X,y) 
    X=Lr.transform(df_X) 
    #print(type(X)) 
    for param in  list(ParameterGrid(params)): 
        Model.set_params(**param)
        SplitMatrix=kfold_split_np(X,y, n_splits)
        aucVector=np.zeros(n_splits)
        counter=0
        for split in SplitMatrix:
            X_train, X_test, y_train, y_test=split[0],split[1],split[2],split[3]
            if Model.__class__.__name__=="XGBClassifier":
                Model.fit(X_train,y_train,eval_set=[(X_test, y_test)],verbose=False) 
            else: 
                Model.fit(X_train,y_train,eval_set=[(X_test, y_test)])
            print("123")
            y_proba = Model.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_proba)
            aucVector[counter]=auc 
            counter+=1
            
        if BestAucScore<np.mean(aucVector):
            BestAucScore=auc
            BestModel=param
    return BestAucScore,BestModel

In [ ]:
X,y=KCrossData() 
BestAucScore,BestModel=XgboostSearch(GetScalePosWeight(y),X,y)
# with open("models/xgb_params.json", "w") as f:
#     json.dump(BestModel, f, indent=4)

123
123
123
123
123
123
123
123
123
123
123
123


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x000001AA83EB94D0>>
Traceback (most recent call last):
  File "C:\Users\Ricardo\Programming\PycharmProjects\ML_Intro-Projects\venv\Lib\site-packages\xgboost\core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 


123


In [ ]:
#BestAucScore,BestModel=LightgbmSearch(X,y)

In [ ]:
import os
os.getcwd()